<a href="https://colab.research.google.com/github/ninamanas/SI_2024/blob/master/Lab_3Nina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [4]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 23.4MB/s]


### Import the required libraries

In [5]:
import pandas as pd

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [16]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor


### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [6]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [7]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [8]:
data.dtypes

,0
Brand,object
Model,object
AccelSec,float64
TopSpeed_KmH,int64
Range_Km,int64
Efficiency_WhKm,int64
FastCharge_KmH,object
RapidCharge,object
PowerTrain,object
PlugType,object


In [10]:
string_columns=[col for col in data.columns if data[col].dtype == 'object' and col!='FastCharge_KmH']
string_columns

['Brand',
 'Model',
 'RapidCharge',
 'PowerTrain',
 'PlugType',
 'BodyStyle',
 'Segment']

In [17]:
label_encoders={}
for col in string_columns:
  label_encoders[col]=LabelEncoder()
  data[col] = label_encoders[col].fit_transform(data[col])

In [18]:
data.replace('-', pd.NA, inplace=True)

In [19]:
data['FastCharge_KmH'] = pd.to_numeric(data['FastCharge_KmH'], errors='coerce')

In [20]:
data['FastCharge_KmH'].fillna(data['FastCharge_KmH'].mean(),inplace=True)

<ipython-input-20-583c15604c59>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['FastCharge_KmH'].fillna(data['FastCharge_KmH'].mean(),inplace=True)


In [21]:
data.isnull().sum()

,0
Brand,0
Model,0
AccelSec,0
TopSpeed_KmH,0
Range_Km,0
Efficiency_WhKm,0
FastCharge_KmH,0
RapidCharge,0
PowerTrain,0
PlugType,0


## Split the dataset for training and testing in ratio 80:20

In [22]:
X = data.drop('PriceEuro',axis=1)
y = data['PriceEuro']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [23]:
decision_regressor=DecisionTreeRegressor(max_depth=10, min_samples_split=5, max_features='sqrt')
decision_regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, max_features='sqrt', min_samples_split=5)

## Predict the outcomes for X test

In [24]:
reg_pred=decision_regressor.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [25]:
decision_mar = mean_absolute_error(y_test, reg_pred)
decision_mse = mean_squared_error(y_test, reg_pred)
decision_r2 = r2_score(y_test, reg_pred)

In [26]:
print("DecisionTreeRegressor Performance:")
print(f"Mean Absolute Error: {decision_mar}")
print(f"Mean Squared Error: {decision_mse}")
print(f"R-squared: {decision_r2}")

DecisionTreeRegressor Performance:
Mean Absolute Error: 10575.218253968254
Mean Squared Error: 276162101.3925265
R-squared: 0.6517667193850591


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [27]:
xgb=XGBRegressor(n_estimators = 100, max_depth=6, learning_rate=0.1, objective="reg:squarederror")
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [28]:
xgb_pred = xgb.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [29]:
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

In [30]:
print("\nXGBRegressor Performance:")
print(f"Mean Absolute Error: {xgb_mae}")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared: {xgb_r2}")


XGBRegressor Performance:
Mean Absolute Error: 7674.605840773809
Mean Squared Error: 229197751.91820925
R-squared: 0.7109875679016113


## Compare the performances of both model for at least three regression metircs

In [31]:
print("\nComparison of Model Performances:")
print(f"DecisionTreeRegressor - MAE: {decision_mar}, MSE: {decision_mse}, R2: {decision_r2}")
print(f"XGBRegressor - MAE: {xgb_mae}, MSE: {xgb_mse}, R2: {xgb_r2}")


Comparison of Model Performances:
DecisionTreeRegressor - MAE: 10575.218253968254, MSE: 276162101.3925265, R2: 0.6517667193850591
XGBRegressor - MAE: 7674.605840773809, MSE: 229197751.91820925, R2: 0.7109875679016113
